# 3. Делаем RAG, который хорошо ищет информацию в документах

## Импорты

In [55]:
import os
import pandas as pd
from chonkie import RecursiveChunker
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client import models
from sentence_transformers import SentenceTransformer
import uuid

tqdm.pandas()

api_key = os.getenv("OPENROUTER_API")


In [56]:
client = QdrantClient("http://localhost:6333")

with open("RAG_Test_Easy.md") as f:
    text = f.read()


## Прогоняем текст, через модель в векторную базу данных

In [57]:
encoder = SentenceTransformer("deepvk/USER2-base")
collection_name = str(uuid.uuid1())

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        )
    )
)

chunker : RecursiveChunker = RecursiveChunker().from_recipe("markdown", lang="en")

points = []

for chunk in tqdm(chunker.chunk(text)):
    points.append(models.PointStruct(
        id=len(points) + 1,
        vector=[
            encoder.encode(chunk.text).tolist(),
            encoder.encode(chunk.text, prompt_name="search_query").tolist()
        ],
        payload={"chunk": chunk.text}
    ))

client.upload_points(collection_name, points)


100%|██████████| 18/18 [00:15<00:00,  1.20it/s]


## Заставляем модель ответить на вопросы

In [58]:
# RAG-агент на LangChain поверх Qdrant
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM через OpenRouter (OpenAI-совместимый API)
llm = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    temperature=0.0,
)

# Промпт для строгого RAG
prompt = ChatPromptTemplate.from_template(
    """Ты — строгий RAG-помощник.
Отвечай только на основе приведённого контекста.
Если в контексте нет ответа, честно скажи об этом.

Контекст:
{context}

Вопрос: {question}

В ответе нужено только число, без единиц измерения, без указания что это проценты:"""
)

def rag_answer(question: str) -> str:
    """Ответ на вопрос только из базы знаний через LangChain."""
    docs = client.query_points(
        collection_name=collection_name, 
        query=[
            encoder.encode(question).tolist(),
            encoder.encode(question, prompt_name='search_query').tolist()
        ],).points
    context = "\n\n".join(d.payload['chunk'] for d in docs)
    messages = prompt.format_messages(context=context, question=question)
    resp = llm.invoke(messages)
    return resp.content

In [59]:
# Прогоняем вопросы из RAG_questions_table.csv через RAG-агента
df_q = pd.read_csv("RAG_questions_table.csv", index_col='row_id')
df_a = pd.read_csv("RAG_answers.csv", index_col='row_id')

# Предполагаем, что столбцы называются 'Question' и 'Anwer'
df_a["Model_Answer"] = df_q["Question"].progress_apply(
    lambda q: rag_answer(str(q))
)

  0%|          | 0/112 [00:00<?, ?it/s]

100%|██████████| 112/112 [03:18<00:00,  1.77s/it]


Ого точность 97 процентов

In [60]:
df_a['Model_Answer'] = pd.to_numeric(df_a['Model_Answer'].str.replace(r"\s+", "", regex=True))

(df_a['Answer'] == df_a['Model_Answer']).sum() / df_a.shape[0]

np.float64(0.9732142857142857)

In [61]:
df_a[(df_a['Answer'] != df_a['Model_Answer'])]

,Answer,Model_Answer
row_id,,
28,9.0,9000000.0
114,10.0,1000.0
120,10.0,10000.0


## Работа над ошибками

In [ ]:
def get_context(question: str):
    docs = client.query_points(
        collection_name=collection_name, 
        query=[
            encoder.encode(question).tolist(),
            encoder.encode(question, prompt_name='search_query').tolist()
        ],).points
    context = "\n\n".join(d.payload['chunk'] for d in docs)
    print(context)

В этом случае модель не угадала с единицами в ответе, а так ответ правильный

In [67]:
print(df_q.loc[28, 'Question'])
print('=' * 200)
get_context(df_q.loc[28, 'Question'])

Сколько миллионов точек данных в секунду могут обрабатывать финансовые торговые агенты?
#### **3.2 Интеграция с ИИ и Агентами / Integration with AI and Agents**

n8n's ability to integrate with AI models and agents amplifies its utility. Workflows can be designed to:

* **Trigger AI analysis:** Automatically send new customer reviews from a CRM to an AI sentiment analysis model and then notify the relevant department if **20% (двадцать процентов)** or more reviews are negative.  
* **Orchestrate AI agents:** Use n8n to sequence actions for multiple AI agents. For instance, an agent could extract data from a document, another agent could summarize it, and a third could then generate a report, all orchestrated by a single n8n workflow. This significantly reduces human intervention in data processing, potentially handling **hundreds (сотни)** of documents per day.  
* **Automate data pipelines for AI training:** Collect and pre-process data from various sources for AI model training, ensu

В этом случае не удачное предложение попала в контекст, и он написал просто тысячу `В них росли **тысячи** разноцветных растений, излучающих тепло и свет. Он насчитал **15 различных видов** фруктов и **25 видов** овощей.`

In [68]:
print(df_q.loc[114, 'Question'])
print('=' * 200)
get_context(df_q.loc[114, 'Question'])

Сколько разноцветных растений росло в садах? Ответ в тысячах
На вершине он увидел карту, выгравированную на камне. Она показывала **3 стрелки**, указывающие в разных направлениях, и **8 символов**, которые нужно было расшифровать. После **4 часов** размышлений, Рустик понял, что ему нужно идти на север, пройти **200 километров**, затем на восток еще **150 километров**.

Следующие **20 дней** были самыми трудными. Температура опускалась до **минус 100 градусов Цельсия** ночью, и поднималась до **20 градусов Цельсия** днем. Он увидел **1000 падающих звезд** за одну ночь. Он преодолел **2 широкие каньона**, каждый глубиной **5 километров**.

На **90-й день** своего путешествия, Рустик наконец-то увидел это\! Прямо перед ним раскинулись Затерянные Сады. Они были огромны, площадью примерно **100 квадратных километров**. В них росли **тысячи** разноцветных растений, излучающих тепло и свет. Он насчитал **15 различных видов** фруктов и **25 видов** овощей.

Рустик быстро вернулся домой. Его п

Тут неправильный ответ в датасете, в вопросе справшивают килограммы, модель написала правильно, но в вопросе в тоннах

За **1 год** Лабуба собрали **10 тонн** еды из новых садов.

In [69]:
print(df_q.loc[120, 'Question'])
print('=' * 200)
get_context(df_q.loc[120, 'Question'])

Сколько киллограм еды собрали Лабуба за 1 год?
На вершине он увидел карту, выгравированную на камне. Она показывала **3 стрелки**, указывающие в разных направлениях, и **8 символов**, которые нужно было расшифровать. После **4 часов** размышлений, Рустик понял, что ему нужно идти на север, пройти **200 километров**, затем на восток еще **150 километров**.

Следующие **20 дней** были самыми трудными. Температура опускалась до **минус 100 градусов Цельсия** ночью, и поднималась до **20 градусов Цельсия** днем. Он увидел **1000 падающих звезд** за одну ночь. Он преодолел **2 широкие каньона**, каждый глубиной **5 километров**.

На **90-й день** своего путешествия, Рустик наконец-то увидел это\! Прямо перед ним раскинулись Затерянные Сады. Они были огромны, площадью примерно **100 квадратных километров**. В них росли **тысячи** разноцветных растений, излучающих тепло и свет. Он насчитал **15 различных видов** фруктов и **25 видов** овощей.

Рустик быстро вернулся домой. Его путешествие зан